In [1]:
from collections import defaultdict
from functools import partial
from pathlib import Path
import shutil
import sys
import time
from typing import Any, Dict, Optional, Tuple
import random
import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import wandb

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn

from modules.agent import Agent
from modules.utils import set_seed, make_env, \
                            get_weight_generators, get_optimizers


ImportError: cannot import name 'make_env' from 'modules.utils' (/home/kukjin/kukjin/Projects/MultiEnvRL/DomainAgnosticRL/modules/utils.py)

In [ ]:
cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DomainAgnosticRL/output/2023-06-27/07-32-46/.hydra/config.yaml"
ckpt_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DomainAgnosticRL/output/2023-06-27/07-32-46/checkpoints/192"

In [ ]:
cfg = OmegaConf.load(cfg_path)
hparams_cfg = cfg.hyperparams
exp_cfg = cfg.experiment

In [4]:
cfg

{'experiment': {'env_ids': ['HalfCheetah-v4'], 'algorithm': 'PPO', 'seed': 42, 'eval_seed': 3142, 'max_episode_steps': 1000, 'num_rollout_steps': 128, 'num_train_envs': 16, 'total_timesteps': 5000000, 'save_ckpt': True, 'num_checkpoints': 50, 'print_interval': 100, 'stop_after_epochs': 500, 'capture_video': True, 'device': 0, 'cuda': True, 'torch_deterministic': True, 'resume': False, 'resume_update_idx': 0, 'resume_dir': 'None'}, 'wandb': {'mode': 'online', 'project': 'DomainAgnosticRL', 'entity': None, 'name': None, 'group': 'single_continuous', 'tags': None, 'notes': None}, 'paths': {'dir': 'output/${now:%Y-%m-%d/%H-%M-%S}', 'log': 'output/${now:%Y-%m-%d/%H-%M-%S}/runs', 'video': 'output/${now:%Y-%m-%d/%H-%M-%S}/videos', 'checkpoints': 'output/${now:%Y-%m-%d/%H-%M-%S}/checkpoints', 'src': 'output/${now:%Y-%m-%d/%H-%M-%S}/src', 'scripts': 'output/${now:%Y-%m-%d/%H-%M-%S}/scripts'}, 'hyperparams': {'encoder_generator': {'type': 'rnn', 'final_activation': 'identity', 'initializer': 'un

In [5]:
EncoderGenerator, PolicyGenerator = get_weight_generators(hparams_cfg)


In [6]:
device = torch.device(f"cuda:{exp_cfg.device}" \
                     if torch.cuda.is_available() and exp_cfg.cuda else "cpu")
        
agent = Agent(hparams_cfg).to(device)
eg = EncoderGenerator(hparams_cfg).to(device)
pg_prob = PolicyGenerator(hparams_cfg).to(device)
pg_mu = PolicyGenerator(hparams_cfg).to(device)
pg_logstd = PolicyGenerator(hparams_cfg).to(device)
pg = (pg_prob, pg_mu, pg_logstd)

agent_optimizer, eg_optimizer, pg_prob_optimizer, \
pg_mu_optimizer, pg_logstd_optimizer = get_optimizers(
                                                hparams_cfg,
                                                agent,
                                                eg, 
                                                pg
                                                )



/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ConfigAttributeError: Missing key weight_decay
    full_key: hyperparams.shared_networks.weight_decay
    object_type=dict

In [ ]:
agent_pt = Path(ckpt_path) / 'agent.pt'
opt_pt = Path(ckpt_path) / 'optimizer.pt'
agent_state_dict = torch.load(agent_pt, map_location=device)
ckpt_opt = torch.load(opt_pt, map_location=device)



In [ ]:
agent.load_state_dict(agent_state_dict['agent'])
eg.load_state_dict(agent_state_dict['eg'])
pg_prob.load_state_dict(agent_state_dict['pg_prob'])
pg_mu.load_state_dict(agent_state_dict['pg_mu'])
pg_logstd.load_state_dict(agent_state_dict['pg_logstd'])
agent_optimizer.load_state_dict(ckpt_opt['agent_optimizer'])
eg_optimizer.load_state_dict(ckpt_opt['eg_optimizer'])
pg_prob_optimizer.load_state_dict(ckpt_opt['pg_prob_optimizer'])
pg_mu_optimizer.load_state_dict(ckpt_opt['pg_mu_optimizer'])
pg_logstd_optimizer.load_state_dict(ckpt_opt['pg_logstd_optimizer'])



In [8]:
env = gym.make("HalfCheetah-v4")
env.action_space

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


Box(-1.0, 1.0, (6,), float32)

In [154]:
vars(pg_logstd_optimizer)

{'defaults': {'lr': 0.001,
  'betas': (0.9, 0.999),
  'eps': 1e-05,
  'weight_decay': 0.9,
  'amsgrad': False,
  'maximize': False,
  'foreach': None,
  'capturable': False,
  'differentiable': False,
  'fused': False},
 '_zero_grad_profile_name': 'Optimizer.zero_grad#Adam.zero_grad',
 'state': defaultdict(dict,
             {Parameter containing:
              tensor([[ 1.3170e-03, -8.8016e-04,  1.4237e-03,  ...,  9.1921e-04,
                        1.5885e-03,  1.0954e-03],
                      [ 9.7353e-04, -5.7393e-04,  1.2929e-03,  ...,  9.3540e-04,
                        1.2644e-03,  1.0379e-03],
                      [ 1.1887e-03, -1.0081e-03,  6.8867e-05,  ..., -8.4719e-05,
                        1.3449e-03,  1.6880e-05],
                      ...,
                      [ 9.6097e-02, -8.4015e-02, -7.0930e-02,  ..., -9.7061e-02,
                        1.1829e-01, -8.8045e-02],
                      [-1.1533e-01,  1.0778e-01,  8.5647e-02,  ...,  1.0730e-01,
                  

In [157]:
print(cfg.experiment.capture_video)
cfg.experiment.capture_video = False
print(cfg.experiment.capture_video)



True
False


In [158]:
# video_path = Path(cfg.paths.video)



In [159]:
# cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DomainAgnosticRL/output/2023-06-22/06-24-16/.hydra/"
# @hydra.main(version_base=None, config_path=cfg_path, config_name="config")
# def main(cfg):
test_envs_returns = dict()
test_envs_lengths = dict()
env_ids = ["HalfCheetah-v4"]
test_different_envs = []
eval_seed=3142
for env_id in env_ids:
    test_env = make_env(cfg, env_id, eval_seed, 0, evaluation=False)()
    test_different_envs.append(test_env)

for i, env in enumerate(test_different_envs):
    next_obs, info = env.reset(seed=eval_seed + i)
    next_obs = torch.Tensor(next_obs).to(device).view(1, -1)
    done = False
    while not done:
        with torch.no_grad():
            action, logprob, _, value = agent.get_action_and_value(eg, pg, env.action_space, next_obs)
        if len(action.shape) > 1:
            action = action.squeeze(0)
            next_obs, reward, truncated, terminated, info = env.step(action.cpu().numpy())
        else:
            next_obs, reward, truncated, terminated, info = env.step(action.cpu().numpy().item())
            
        done = np.logical_or(truncated, terminated)
        next_obs = torch.Tensor(next_obs).to(device).view(1, -1)
        if done:
            print(f"Evaluation. env_name:{env_ids[i]}, episodic_return={info['episode']['r']}")
            test_envs_returns[env_ids[i]+'/eval'] = info["episode"]["r"]
            test_envs_lengths[env_ids[i]+'/eval'] = info["episode"]["l"]


Evaluation. env_name:HalfCheetah-v4, episodic_return=[26.183979]


In [23]:
-223148.31 / 1000

-223.14831

In [160]:
print(info)

{'x_position': 16.729382928337053, 'x_velocity': -0.0707944977606445, 'reward_run': -0.0707944977606445, 'reward_ctrl': -0.1901054859161377, 'episode': {'r': array([26.183979], dtype=float32), 'l': array([1000], dtype=int32), 't': array([3.], dtype=float32)}}


In [161]:
vars(eg)

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('final_act_func', Identity()),
              ('rnn', GRU(1, 64, bias=False, batch_first=True))]),
 'input_shared_dim': 64,
 'output_shared_dim': 32,
 'final_act': 'identity',
 'initializer': 'uniform'}

In [162]:
print(eg.rnn.weight_ih_l0.shape)
print(eg.rnn.weight_hh_l0.shape)

torch.Size([192, 1])
torch.Size([192, 64])


In [163]:
weight_list = []
for name in eg.rnn.named_parameters():
    print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])

('weight_ih_l0', Parameter containing:
tensor([[-3.5995e-01],
        [ 2.0376e-01],
        [ 1.7949e-01],
        [-1.2876e-01],
        [ 4.8074e-01],
        [ 4.3987e-01],
        [ 1.5566e-01],
        [-4.4247e-01],
        [ 5.7117e-01],
        [-3.5994e-01],
        [-7.6881e-01],
        [ 2.1692e-01],
        [ 8.6583e-02],
        [ 1.0360e-01],
        [-1.2688e-02],
        [-5.3005e-01],
        [ 2.0756e-01],
        [ 3.6554e-01],
        [-1.2384e-01],
        [-1.8567e-01],
        [-9.7427e-01],
        [ 3.8657e-01],
        [ 2.2039e-01],
        [-4.6247e-01],
        [ 2.3933e-01],
        [-6.4943e-01],
        [-7.7812e-01],
        [ 8.7602e-01],
        [ 2.0970e-01],
        [ 2.6336e-01],
        [ 1.2094e-01],
        [-8.7322e-01],
        [ 6.6801e-01],
        [-3.7813e-02],
        [-3.6369e-01],
        [ 4.3245e-01],
        [ 3.4521e-01],
        [-2.3736e-01],
        [-6.5090e-01],
        [ 4.3392e-01],
        [ 4.8125e-02],
        [ 3.1900e-

In [164]:
for weight in weight_list:
    print(weight.shape)

torch.Size([192, 1])
torch.Size([192, 64])


In [165]:
encoder_weights = eg(next_obs)

In [166]:
encoder_weights

tensor([[[ 3.9306e-01,  5.9361e-02, -5.3076e-01,  ...,  3.9910e-01,
           2.9995e-01, -4.2108e-01],
         [ 2.2757e-02, -1.9021e-01, -5.6087e-01,  ..., -4.9772e-03,
          -2.0939e-03, -2.5486e-02],
         [-8.1892e-02, -7.1712e-01, -7.3657e-01,  ...,  1.3190e-01,
          -8.1026e-01, -9.5477e-01],
         ...,
         [-1.0557e-02,  2.2409e-01,  3.0898e-01,  ..., -1.5555e-01,
           1.3414e-01,  6.7304e-01],
         [ 6.2115e-02,  4.6943e-01,  8.1827e-02,  ..., -1.1191e-01,
          -8.2498e-02, -9.1482e-02],
         [-4.0336e-04, -7.8773e-01, -7.3028e-01,  ...,  9.5581e-01,
          -1.2295e-01, -7.1959e-01]]], device='cuda:0',
       grad_fn=<PermuteBackward0>)

In [167]:
import torch.nn.functional as F


In [168]:
def bmm_input(b_weight, b_input):
    bmm = torch.einsum('bhf, bf -> bh', b_weight, b_input)
    return bmm

def bmm_output(b_weight, b_input):
    bmm = torch.einsum('bfa, bf -> ba', b_weight, b_input)
    return bmm


In [169]:
cfg.hyperparams.shared_networks

{'input_shared_dim': 64, 'hidden_dim': 64, 'output_shared_dim': 32, 'num_hidden_layers': 1, 'type': 'mlp', 'activation': 'gelu', 'residual': False, 'optimizer': 'adam', 'lr': 0.001}

In [170]:
hidden = F.gelu(bmm_input(encoder_weights, next_obs)) 


In [171]:
agent.shared_net.network[4].weight

Parameter containing:
tensor([[ 0.0290,  0.1497, -0.0402,  ..., -0.1390,  0.2915,  0.1146],
        [ 0.0216,  0.0337, -0.0908,  ..., -0.1287,  0.1125,  0.0011],
        [ 0.0645, -0.0775, -0.1019,  ..., -0.0119,  0.0129,  0.0053],
        ...,
        [ 0.0192,  0.1980,  0.2608,  ..., -0.0862, -0.0202, -0.0064],
        [-0.0412,  0.0476,  0.2147,  ...,  0.0629, -0.0922, -0.0013],
        [ 0.0458, -0.0643, -0.2538,  ...,  0.0827, -0.1739,  0.0155]],
       device='cuda:0', requires_grad=True)

In [172]:
shared_feature = F.gelu(agent.shared_net(hidden)) 
act_dim = 6
a_mu_feature = agent.actor_mu(shared_feature)
a_logstd_feature = agent.actor_logstd(shared_feature)


In [173]:
shared_feature

tensor([[ 1.7278, -0.1615,  1.0679, -0.1479, -0.1686, -0.1208,  0.5242, -0.1400,
         -0.1413,  0.2619, -0.1700,  0.1722, -0.1653, -0.1626,  1.2711, -0.1563,
          1.0409, -0.0676, -0.1321,  0.5021, -0.1691,  0.2844, -0.1378,  0.8260,
         -0.0733, -0.0853, -0.1389, -0.1446,  0.9482, -0.1422, -0.1408, -0.1678]],
       device='cuda:0', grad_fn=<GeluBackward0>)

In [174]:
a_logstd_feature

tensor([[-0.0876,  0.0543, -0.0018, -0.0328, -0.0937, -0.0316,  0.0800, -0.0195,
         -0.0065, -0.0040,  0.0167, -0.0583,  0.0055,  0.0837,  0.0566, -0.0868,
         -0.0094, -0.0186,  0.0082,  0.0127,  0.0280, -0.0243, -0.0050, -0.0065,
         -0.0891,  0.0285, -0.0025, -0.0166, -0.0555,  0.0226, -0.0980,  0.0397]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [175]:
weight_list = []
for name in pg_mu.rnn.named_parameters():
    # print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])
        
for weight in weight_list:
    print(torch.round(weight, decimals=4))
    print(torch.norm(weight, 'fro'))
    

tensor([[ 1.2600e-02,  1.1200e-02,  2.4900e-02,  ...,  1.0400e-02,
          9.4000e-03, -4.0000e-04],
        [ 1.0600e-02,  8.7000e-03,  1.7400e-02,  ...,  9.0000e-03,
          7.6000e-03, -1.0000e-04],
        [ 2.6200e-02,  2.2600e-02,  5.9300e-02,  ...,  1.8400e-02,
          1.5300e-02,  0.0000e+00],
        ...,
        [ 3.5700e-02,  4.8800e-02,  3.7400e-02,  ..., -2.0500e-02,
         -2.8500e-02,  1.0000e-03],
        [ 2.8800e-02,  4.8800e-02,  4.5000e-02,  ..., -3.7500e-02,
         -4.5100e-02, -6.0000e-04],
        [-2.5000e-03, -1.1000e-03,  1.8000e-03,  ...,  2.3000e-03,
          4.0000e-04,  2.0000e-04]], device='cuda:0', grad_fn=<RoundBackward1>)
tensor(1.4699, device='cuda:0', grad_fn=<NormBackward1>)
tensor([[-0.0101, -0.0095, -0.0166,  ...,  0.0016,  0.0008,  0.0010],
        [-0.0090, -0.0088, -0.0134,  ...,  0.0027,  0.0020,  0.0005],
        [-0.0187, -0.0155, -0.0343,  ..., -0.0031, -0.0037,  0.0005],
        ...,
        [-0.0918, -0.0785, -0.1146,  ...,  0.

In [176]:
weight_list = []
for name in pg_logstd.rnn.named_parameters():
    # print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])
        
for weight in weight_list:
    print(torch.round(weight, decimals=4))
    print(torch.norm(weight, 'fro'))
    

tensor([[ 1.3000e-03, -9.0000e-04,  1.4000e-03,  ...,  9.0000e-04,
          1.6000e-03,  1.1000e-03],
        [ 1.0000e-03, -6.0000e-04,  1.3000e-03,  ...,  9.0000e-04,
          1.3000e-03,  1.0000e-03],
        [ 1.2000e-03, -1.0000e-03,  1.0000e-04,  ..., -1.0000e-04,
          1.3000e-03,  0.0000e+00],
        ...,
        [ 9.6100e-02, -8.4000e-02, -7.0900e-02,  ..., -9.7100e-02,
          1.1830e-01, -8.8000e-02],
        [-1.1530e-01,  1.0780e-01,  8.5600e-02,  ...,  1.0730e-01,
         -1.3990e-01,  1.0560e-01],
        [ 9.2700e-02, -8.0000e-02, -6.3000e-02,  ..., -8.7700e-02,
          1.1320e-01, -8.2200e-02]], device='cuda:0', grad_fn=<RoundBackward1>)
tensor(3.1196, device='cuda:0', grad_fn=<NormBackward1>)
tensor([[ 0.0019, -0.0022, -0.0078,  ..., -0.0089,  0.0019, -0.0090],
        [ 0.0015, -0.0019, -0.0055,  ..., -0.0063,  0.0013, -0.0063],
        [-0.0014,  0.0013,  0.0021,  ...,  0.0026, -0.0017,  0.0026],
        ...,
        [-0.0129,  0.0107,  0.0032,  ...,  0.

In [177]:
# generate policy weights
a_mean_weights = pg_mu(act_dim, a_mu_feature)
a_logstd_weights = pg_logstd(act_dim, a_logstd_feature)
print(a_mean_weights.shape)
print(a_logstd_weights.shape)



torch.Size([1, 32, 6])
torch.Size([1, 32, 6])


In [178]:
a_mean_weights

tensor([[[-1.5163e-01, -1.8254e-01, -1.8426e-01, -1.8043e-01, -1.7676e-01,
          -1.7423e-01],
         [-1.6324e-01, -2.1151e-01, -2.2447e-01, -2.2672e-01, -2.2612e-01,
          -2.2510e-01],
         [-3.2111e-01, -3.6102e-01, -3.5458e-01, -3.4563e-01, -3.3995e-01,
          -3.3689e-01],
         [ 8.2902e-03,  9.4317e-03,  8.7324e-03,  7.7923e-03,  7.0363e-03,
           6.5175e-03],
         [-2.6845e-01, -3.2466e-01, -3.3367e-01, -3.3302e-01, -3.3124e-01,
          -3.2992e-01],
         [-9.9507e-03, -1.0407e-02, -8.8239e-03, -7.2199e-03, -6.0338e-03,
          -5.2545e-03],
         [-2.1359e-01, -2.7251e-01, -2.8937e-01, -2.9396e-01, -2.9498e-01,
          -2.9504e-01],
         [-1.1027e-02, -1.2106e-02, -1.0801e-02, -9.2982e-03, -8.1362e-03,
          -7.3530e-03],
         [-5.5168e-02, -1.3401e-01, -1.8663e-01, -2.1663e-01, -2.3276e-01,
          -2.4121e-01],
         [-1.0200e-02, -1.1268e-02, -1.0056e-02, -8.6275e-03, -7.5132e-03,
          -6.7586e-03],
         [

In [179]:
a_logstd_weights

tensor([[[ 0.0574,  0.0774,  0.0822,  0.0817,  0.0799,  0.0783],
         [-0.0530, -0.0718, -0.0766, -0.0765, -0.0751, -0.0737],
         [-0.0436, -0.0688, -0.0829, -0.0908, -0.0950, -0.0973],
         [ 0.0568,  0.0885,  0.1059,  0.1152,  0.1201,  0.1226],
         [ 0.1084,  0.1711,  0.2051,  0.2227,  0.2317,  0.2361],
         [ 0.0743,  0.1149,  0.1365,  0.1477,  0.1533,  0.1562],
         [-0.0577, -0.0776, -0.0822, -0.0816, -0.0797, -0.0780],
         [ 0.0503,  0.0782,  0.0934,  0.1016,  0.1058,  0.1081],
         [-0.0517, -0.0793, -0.0937, -0.1011, -0.1048, -0.1067],
         [-0.0681, -0.1072, -0.1289, -0.1407, -0.1469, -0.1501],
         [ 0.0273,  0.0401,  0.0459,  0.0485,  0.0497,  0.0501],
         [ 0.0453,  0.0610,  0.0647,  0.0644,  0.0630,  0.0617],
         [-0.0576, -0.0890, -0.1057, -0.1144, -0.1189, -0.1212],
         [-0.0571, -0.0776, -0.0829, -0.0829, -0.0815, -0.0800],
         [-0.0527, -0.0710, -0.0754, -0.0750, -0.0734, -0.0718],
         [ 0.1082,  0.169

In [180]:
torch.norm(a_logstd_weights, 'fro')

tensor(1.3929, device='cuda:0', grad_fn=<NormBackward1>)

In [181]:
a_mu = bmm_output(a_mean_weights, a_mu_feature)
a_logstd = bmm_output(a_logstd_weights, a_logstd_feature)


In [182]:
print(a_mu)
print(a_logstd)


tensor([[-0.7568, -0.7167, -0.5972, -0.5122, -0.4627, -0.4358]],
       device='cuda:0', grad_fn=<ViewBackward0>)
tensor([[-0.0703, -0.1023, -0.1158, -0.1210, -0.1228, -0.1231]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [183]:
from torch.distributions import Normal
# a_logstd = torch.clamp(a_logstd , min=-20, max=1)
print(a_logstd)
actor_std = torch.exp(a_logstd)
print(actor_std)


tensor([[-0.0703, -0.1023, -0.1158, -0.1210, -0.1228, -0.1231]],
       device='cuda:0', grad_fn=<ViewBackward0>)
tensor([[0.9321, 0.9028, 0.8906, 0.8860, 0.8845, 0.8841]], device='cuda:0',
       grad_fn=<ExpBackward0>)


In [184]:
import math
print(math.exp(1)**3)

20.085536923187664


In [185]:
probs = Normal(a_mu, actor_std)
print(probs)
if action is None:
    action = probs.sample()

Normal(loc: torch.Size([1, 6]), scale: torch.Size([1, 6]))


In [186]:
print(action)

tensor([ 0.5266,  0.8792, -0.3001, -0.7534, -0.2507,  0.3608], device='cuda:0')


In [187]:
ctrl_cost = -torch.sum(action * action) * 0.1

In [188]:
ctrl_cost

tensor(-0.1901, device='cuda:0')

In [191]:
seed = 42
different_envs = []
for env_id in env_ids:
    make_env_func_list = []
    for i in range(exp_cfg.num_train_envs):
        make_env_func_list.append(make_env(cfg, env_id, seed + i, i))
    train_envs = gym.vector.SyncVectorEnv(make_env_func_list)
    different_envs.append(train_envs)

envs=different_envs[0]

In [192]:
envs_storages = dict()
for i, envs in enumerate(different_envs):
    obs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs) \
        + envs.single_observation_space.shape).to(device)
    actions = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs) \
        + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs)).to(device)
    rewards = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs)).to(device)
    dones = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs)).to(device)
    values = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_train_envs)).to(device)
    storage = dict()
    storage["obs"] = obs
    storage['actions'] = actions
    storage['logprobs'] = logprobs
    storage['rewards'] = rewards
    storage['dones'] = dones
    storage['values'] = values
    envs_storages[i] = storage


In [193]:
hparams_cfg = cfg.hyperparams
exp_cfg = cfg.experiment
batch_size = int(exp_cfg.num_train_envs * exp_cfg.num_rollout_steps)
minibatch_size = int(batch_size // hparams_cfg.training.num_minibatches)
# environment initialization
global_step = 0
start_time = time.time()
next_obsv = []
next_donev = []

# ! Parallize? 
for envs in different_envs:
    next_obs, info = envs.reset()
    next_obs = torch.Tensor(next_obs).to(device)
    next_done = torch.zeros(exp_cfg.num_train_envs).to(device)
    next_obsv.append(next_obs)
    next_donev.append(next_done)

In [194]:
total_num_updates = exp_cfg.total_timesteps // batch_size
start_update_idx = 1 

In [195]:
from gymnasium.spaces import Box

In [196]:
for update_idx in range(start_update_idx, total_num_updates + 1):
    for step in range(0, exp_cfg.num_rollout_steps):
        global_step += 1 * exp_cfg.num_train_envs
        envs_returns = dict()
        envs_lengths = dict()
        for i, envs in enumerate(different_envs):
            envs_storages[i]["obs"][step] = next_obsv[i]
            envs_storages[i]["dones"][step] = next_donev[i]
            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(eg, pg, envs.single_action_space, next_obsv[i])

            envs_storages[i]["values"][step] = value.flatten()
            envs_storages[i]["actions"][step] = action
            envs_storages[i]["logprobs"][step] = logprob

            # TRY NOT TO MODIFY: execute the game and log data.
            next_obs, reward, truncated, terminated, infos = envs.step(action.cpu().numpy())
            done = np.logical_or(truncated, terminated)
            envs_storages[i]["rewards"][step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(done).to(device)
            next_obsv[i] = next_obs
            next_donev[i] = next_done
                
            # Only logger.info when at least 1 env is done
            if "final_info" not in infos:
                continue

            for info in infos["final_info"]:
                # Skip the envs that are not done
                if info is None:
                    continue
                print((f"[{update_idx}/{total_num_updates}] Train. env_name:{env_ids[i]}, global_step={global_step}, episodic_return={info['episode']['r']}"))
                envs_returns[env_ids[i]+'/train'] = info["episode"]["r"]
                envs_lengths[env_ids[i]+'/train'] = info["episode"]["l"]
        
    # bootstrap value if not done
    with torch.no_grad():
        for i, envs in enumerate(different_envs):
            next_value = agent.get_value(eg, next_obsv[i]).reshape(1, -1)
            envs_storages[i]['advantages'] = torch.zeros_like(envs_storages[i]['rewards']).to(device)
            lastgaelam = 0
            for t in reversed(range(exp_cfg.num_rollout_steps)):
                if t == exp_cfg.num_rollout_steps - 1:
                    nextnonterminal = 1.0 - next_donev[i]
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - envs_storages[i]["dones"][t + 1]
                    nextvalues =  envs_storages[i]['values'][t + 1]
                delta = envs_storages[i]['rewards'][t] + hparams_cfg.training.gamma * nextvalues * nextnonterminal - envs_storages[i]['values'][t]
                envs_storages[i]['advantages'][t] = lastgaelam = delta + hparams_cfg.training.gamma * hparams_cfg.training.gae_lambda * nextnonterminal * lastgaelam
            envs_storages[i]['returns'] = envs_storages[i]['advantages'] + envs_storages[i]['values']

    # flatten the batch
    for i, envs in enumerate(different_envs):
        envs_storages[i]["b_obs"] = envs_storages[i]["obs"].reshape((-1,) + envs.single_observation_space.shape)
        envs_storages[i]['b_logprobs'] = envs_storages[i]['logprobs'].reshape(-1)
        envs_storages[i]['b_actions'] = envs_storages[i]['actions'].reshape((-1,) + envs.single_action_space.shape)
        envs_storages[i]['b_advantages'] = envs_storages[i]['advantages'].reshape(-1)
        envs_storages[i]['b_returns'] = envs_storages[i]['returns'].reshape(-1)
        envs_storages[i]['b_values'] = envs_storages[i]['values'].reshape(-1)

    # Optimizing the policy and value network
    b_inds = np.arange(batch_size)
    clipfracs = []
    with torch.autograd.detect_anomaly():
        for epoch in range(hparams_cfg.training.update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, batch_size, minibatch_size):
                total_envs_loss = 0.0
                for i, envs in enumerate(different_envs):
                    end = start + minibatch_size
                    mb_inds = b_inds[start:end]
                    mb_obs = envs_storages[i]['b_obs'][mb_inds]
                    mb_actions = envs_storages[i]['b_actions'][mb_inds]
                    mb_logprobs = envs_storages[i]['b_logprobs'][mb_inds]
                    
                    _, newlogprob, entropy, newvalue = agent.get_action_and_value(eg, pg, envs.single_action_space, mb_obs, mb_actions)
                    
                    logratio = newlogprob - mb_logprobs
                    ratio = logratio.exp()

                    with torch.no_grad():
                        # calculate approx_kl http://joschu.net/blog/kl-approx.html
                        old_approx_kl = (-logratio).mean()
                        approx_kl = ((ratio - 1) - logratio).mean()
                        clipfracs += [((ratio - 1.0).abs() > hparams_cfg.training.clip_coef).float().mean().item()]

                    mb_advantages = envs_storages[i]['b_advantages'][mb_inds]
                    if hparams_cfg.training.norm_adv:
                        mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                    # Policy loss
                    pg_loss1 = -mb_advantages * ratio
                    pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - hparams_cfg.training.clip_coef, 1 + hparams_cfg.training.clip_coef)
                    pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                    # Value loss
                    mb_returns = envs_storages[i]['b_returns'][mb_inds].view(-1)
                    mb_values = envs_storages[i]['b_values'][mb_inds].view(-1)
                    newvalue = newvalue.view(-1)
                    if hparams_cfg.training.clip_vloss:
                        v_loss_unclipped = (newvalue - mb_returns) ** 2
                        v_clipped = mb_values + torch.clamp(
                            newvalue - mb_values,
                            -hparams_cfg.training.clip_coef,
                            hparams_cfg.training.clip_coef,
                        )
                        v_loss_clipped = (v_clipped - mb_returns) ** 2
                        v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                        v_loss = 0.5 * v_loss_max.mean()
                    else:
                        v_loss = 0.5 * ((newvalue - mb_returns) ** 2).mean()

                    # regularization term
                    if isinstance(envs.single_action_space, Box):
                        pg_logstd_weights = agent.get_policy_weights('logstd')
                        constraint = torch.norm(pg_logstd_weights, 'fro')
                    else:
                        constraint = 0.0
                        
                    entropy_loss = entropy.mean()
                    loss = pg_loss - hparams_cfg.training.ent_coef * entropy_loss + v_loss * hparams_cfg.training.vf_coef + hparams_cfg.training.const_coef * constraint
                    
                    total_envs_loss += loss
                agent_optimizer.zero_grad()
                if not hparams_cfg.encoder_generator.type == 'random':
                    eg_optimizer.zero_grad()
                if not hparams_cfg.policy_generator.type == 'random':
                    pg_prob_optimizer.zero_grad()
                    pg_mu_optimizer.zero_grad()
                    pg_logstd_optimizer.zero_grad()
                    
                total_envs_loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), hparams_cfg.training.max_grad_norm)
                nn.utils.clip_grad_norm_(pg_prob.parameters(), hparams_cfg.training.max_grad_norm)
                nn.utils.clip_grad_norm_(pg_mu.parameters(), hparams_cfg.training.max_grad_norm)
                nn.utils.clip_grad_norm_(pg_logstd.parameters(), hparams_cfg.training.max_grad_norm)
                agent_optimizer.step()
                if not hparams_cfg.encoder_generator.type == 'random':
                    eg_optimizer.step()
                if not hparams_cfg.policy_generator.type == 'random':
                    pg_prob_optimizer.step()
                    pg_mu_optimizer.step()
                    pg_logstd_optimizer.step()
            
        




/tmp/ipykernel_1169713/2841984538.py:65: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[-137.65378]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[77.82583]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[61.676376]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[109.31848]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[175.85733]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[80.872116]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[-109.15917]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[110.38599]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[101.181946]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[123.98587]
[8/2441] Train. env_name:HalfCheetah-v4, global_step=16000, episodic_return=[96.142105]
[8/2441] Train. env_name:HalfC

/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/torch/autograd/__init__.py:197: UserWarning: Error detected in PowBackward0. Traceback of forward call that caused the error:
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "/home/kukjin/.conda/envs/test_env/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 21

RuntimeError: Function 'PowBackward0' returned nan values in its 0th output.

In [258]:
print(_)
print(newlogprob)
print(entropy)
print(newvalue)

tensor([[-1.6876e+00, -2.9001e-01, -5.0594e-01, -1.0681e+00, -6.5456e-01,
         -9.2536e-01],
        [ 4.1935e-02, -2.8159e-01, -6.8365e-01, -7.8825e-01, -1.0996e+00,
         -4.9410e-01],
        [-1.3242e-01, -1.0658e+00, -5.5579e-01, -2.2808e-01,  1.4009e-02,
         -3.4243e-01],
        [ 2.7922e-01, -8.5107e-02, -2.6377e-01, -4.4164e-01,  6.6250e-02,
         -2.7741e-01],
        [-1.8321e-01, -3.2745e-01, -8.9123e-02, -6.2524e-01, -5.9355e-01,
         -5.7394e-01],
        [ 3.9740e-01, -6.0938e-01, -9.0119e-02, -5.8028e-01, -6.8984e-01,
         -2.0470e-01],
        [-1.9705e-01, -6.6199e-01,  8.3431e-02, -7.0942e-01, -1.0204e+00,
         -4.4059e-01],
        [-1.6419e-01,  7.6270e-01,  2.1683e-01,  5.0673e-01,  1.1489e-01,
          3.7291e-01],
        [-8.6378e-02, -7.6238e-02, -6.9477e-01, -6.0384e-01,  2.2757e-02,
          5.1313e-02],
        [-2.6100e-01,  1.1949e-02, -3.4898e-01, -1.2358e+00, -4.6363e-01,
         -6.1297e-01],
        [-5.7697e-01, -1.1691e

In [197]:
_, newlogprob, entropy, newvalue = agent.get_action_and_value(eg, pg, envs.single_action_space, mb_obs, mb_actions)


In [199]:
entropy

tensor([ 3.6423e+00,  1.0643e+00,  2.9762e+00, -4.8352e-01,  1.5993e+00,
         4.6470e-01,  4.7278e+00,  3.0785e-01,  1.3800e+00,  3.9936e+00,
         2.4441e+00,  4.7267e+00,  1.5980e+00,         inf,  3.8841e+00,
        -3.6774e-01,  3.6127e+00, -5.9743e-01,  3.7402e-01,  3.2497e+00,
        -4.1616e-04,  1.7012e+01,  3.3833e+00,  1.9770e+00,  2.1737e+00,
         1.3715e+00,  4.7965e+00,  2.7322e+00,  2.2442e+00,  3.0049e+00,
         1.8450e+00,  4.6625e+00,  1.4262e+00,  4.6511e+00,  1.5334e+00,
         1.8749e+00,  2.2364e+02, -2.9206e-01, -4.1040e-01,  3.7531e+00,
         1.3518e-01,  2.7406e+00,  3.1443e+00,  5.1068e+00,  4.4364e+00,
        -5.8108e-02,  1.0386e+01, -1.4616e-01,  2.3983e+00,  2.4690e+00,
        -2.9401e-01,  5.3515e+00,  2.9356e+00,  3.6295e+00, -3.2151e-01,
         3.8052e+00,  2.0066e+00,  4.1338e+00,  1.2855e-01,  2.9131e+00,
         1.0963e+00,  4.2692e+00,  3.6368e+00,  4.1375e+00], device='cuda:0',
       grad_fn=<SumBackward1>)

In [200]:
weight_list = []
for name in eg.rnn.named_parameters():
    print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])
        

('weight_ih_l0', Parameter containing:
tensor([[-0.3425],
        [ 0.2175],
        [ 0.1945],
        [-0.1123],
        [ 0.4935],
        [ 0.4947],
        [ 0.1776],
        [-0.4329],
        [ 0.5309],
        [-0.3659],
        [-0.7591],
        [ 0.2273],
        [ 0.0773],
        [ 0.1015],
        [-0.0349],
        [-0.5699],
        [ 0.3019],
        [ 0.3741],
        [-0.1216],
        [-0.2338],
        [-1.0201],
        [ 0.3737],
        [ 0.2037],
        [-0.4656],
        [ 0.2227],
        [-0.6922],
        [-0.7975],
        [ 0.8552],
        [ 0.1796],
        [ 0.2775],
        [ 0.1160],
        [-0.8690],
        [ 0.6610],
        [-0.0779],
        [-0.3784],
        [ 0.4582],
        [ 0.3442],
        [-0.2117],
        [-0.7058],
        [ 0.3672],
        [ 0.0523],
        [ 0.3099],
        [-0.1015],
        [-0.3699],
        [-0.0847],
        [-0.0741],
        [-0.0351],
        [ 0.0095],
        [-0.1363],
        [ 0.0687],
        [-0

In [270]:
encoder_weights = eg(mb_obs)
encoder_weights

tensor([[[-9.6305e-02, -3.4317e-02, -1.8318e-02,  ...,  3.7719e-01,
           3.8061e-01,  3.9319e-01],
         [-7.8323e-03,  7.1378e-02,  1.8394e-01,  ...,  5.9854e-01,
           5.2567e-01,  5.7722e-01],
         [ 4.3377e-02,  1.6017e-01,  3.1614e-01,  ..., -1.5770e-01,
           2.2733e-01, -3.4302e-03],
         ...,
         [ 3.9049e-03, -8.2931e-02, -2.3419e-01,  ..., -9.5359e-01,
          -8.6557e-01, -9.1241e-01],
         [-4.2485e-02, -1.5709e-01, -3.2581e-01,  ...,  7.7103e-01,
           7.3797e-01,  6.2775e-01],
         [ 1.1833e-02,  1.6669e-01,  2.4066e-01,  ..., -1.4946e-01,
          -1.7908e-01, -1.7950e-01]],

        [[-8.9463e-02,  1.8884e-01,  1.3842e-01,  ...,  2.4237e-01,
           3.1166e-02, -3.6600e-01],
         [-7.2009e-03,  8.0043e-02,  1.3435e-02,  ..., -4.2440e-02,
          -4.0265e-02, -3.9429e-02],
         [ 3.9350e-02,  7.7745e-02, -8.7461e-02,  ..., -8.7603e-01,
          -9.2164e-01, -9.3625e-01],
         ...,
         [ 3.6031e-03, -1

In [271]:
hidden = F.gelu(bmm_input(encoder_weights, mb_obs)) 
hidden

tensor([[-1.5452e-01, -1.0605e-01,  1.0976e+00,  ...,  2.5532e+00,
         -1.7848e-02,  6.5941e-02],
        [-3.9008e-02, -1.6133e-01, -9.6329e-02,  ...,  8.5313e-01,
          5.7788e-01,  1.0745e+00],
        [-3.5387e-03, -1.1729e-01, -6.4272e-03,  ...,  7.9056e-01,
          4.8144e-02,  4.3980e+00],
        ...,
        [ 9.1732e-02,  9.7270e-01,  4.3717e+00,  ..., -0.0000e+00,
          3.7123e+00, -2.9033e-03],
        [-3.7320e-02,  3.3622e-02,  1.8246e+00,  ..., -1.6226e-01,
         -1.6680e-01, -6.5913e-03],
        [-1.5257e-01,  3.4245e+00,  2.6523e+00,  ..., -2.8526e-03,
          5.1120e-01,  9.7053e-01]], device='cuda:0', grad_fn=<GeluBackward0>)

In [272]:
agent.shared_net.network[4].weight


Parameter containing:
tensor([[ 0.0482,  0.1560, -0.0581,  ..., -0.1536,  0.2746,  0.1223],
        [ 0.0434,  0.0372, -0.0846,  ..., -0.1242,  0.1449, -0.0149],
        [ 0.0547, -0.0948, -0.1496,  ..., -0.0456,  0.0207, -0.0051],
        ...,
        [ 0.0417,  0.2032,  0.2701,  ..., -0.0850,  0.0067, -0.0274],
        [-0.0197,  0.0768,  0.2361,  ...,  0.0594, -0.1019,  0.0146],
        [ 0.0371, -0.0631, -0.2583,  ...,  0.0852, -0.1967,  0.0046]],
       device='cuda:0', requires_grad=True)

In [273]:
shared_feature = F.gelu(agent.shared_net(hidden)) 
act_dim = 6
a_mu_feature = agent.actor_mu(shared_feature)
a_logstd_feature = agent.actor_logstd(shared_feature)
shared_feature



tensor([[ 5.7184e-01, -8.5013e-02, -1.2932e-01,  ..., -1.6592e-01,
         -1.6544e-01, -1.6723e-01],
        [-1.1412e-01,  8.3058e-02, -1.0024e-01,  ...,  1.6613e-01,
          2.6887e-02, -1.2374e-01],
        [ 3.9957e-01, -1.3660e-01, -1.3576e-01,  ...,  2.2345e-01,
         -1.3580e-02, -1.4605e-01],
        ...,
        [ 3.5288e+00, -4.8704e-05, -8.6523e-02,  ..., -4.5444e-06,
         -4.0487e-02, -5.1511e-02],
        [ 1.8829e-01, -1.3893e-01,  4.0225e-01,  ...,  8.9520e-03,
          4.2043e-02, -1.6505e-01],
        [ 2.6164e+00, -3.4221e-03, -7.1891e-02,  ..., -6.3710e-04,
         -1.6991e-01, -1.2869e-01]], device='cuda:0', grad_fn=<GeluBackward0>)

In [275]:
a_logstd_feature


tensor([[-0.2315,  0.1859,  0.0063,  ...,  0.0326, -0.2606,  0.0295],
        [-0.2275,  0.1917,  0.0500,  ...,  0.0757, -0.2579,  0.0660],
        [-0.2177,  0.1772,  0.0267,  ...,  0.0518, -0.2463,  0.0479],
        ...,
        [-0.1817,  0.1499,  0.0336,  ...,  0.0555, -0.2002,  0.0551],
        [-0.1718,  0.1381,  0.0543,  ...,  0.0755, -0.1946,  0.0768],
        [-0.1968,  0.1610,  0.0240,  ...,  0.0470, -0.2173,  0.0455]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [276]:
weight_list = []
for name in pg_mu.rnn.named_parameters():
    # print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])
        
for weight in weight_list:
    print(torch.round(weight, decimals=4))
    print(torch.norm(weight, 'fro'))
    

tensor([[-1.0000e-04, -6.0000e-04,  1.0600e-02,  ..., -2.0000e-04,
          1.0000e-03, -1.0000e-04],
        [-1.0000e-04, -7.0000e-04,  1.6000e-03,  ...,  1.6000e-03,
          2.8000e-03, -3.0000e-04],
        [ 2.1200e-02,  1.6700e-02,  5.6400e-02,  ...,  1.4800e-02,
          1.4300e-02,  9.0000e-04],
        ...,
        [ 3.4600e-02,  4.7800e-02,  3.8600e-02,  ..., -2.0600e-02,
         -2.7900e-02, -1.5000e-03],
        [ 2.9700e-02,  5.0500e-02,  4.9500e-02,  ..., -3.7600e-02,
         -4.5800e-02, -1.7000e-03],
        [-4.0000e-04,  8.0000e-04,  4.0000e-04,  ..., -2.0000e-04,
         -6.0000e-04, -5.0000e-04]], device='cuda:0', grad_fn=<RoundBackward1>)
tensor(1.4545, device='cuda:0', grad_fn=<NormBackward1>)
tensor([[ 1.0000e-04, -6.0000e-04, -1.6000e-03,  ...,  1.1000e-03,
          1.0000e-03, -0.0000e+00],
        [-2.0000e-04,  0.0000e+00,  3.0000e-04,  ..., -2.0000e-04,
         -3.0000e-04,  0.0000e+00],
        [-1.6500e-02, -1.5600e-02, -3.1100e-02,  ...,  2.2000e

In [277]:
weight_list = []
for name in pg_logstd.rnn.named_parameters():
    # print(name)
    if 'weight' in name[0]:
        weight_list.append(name[1])
        
for weight in weight_list:
    print(weight_list)
    print(torch.round(weight, decimals=4))
    print(torch.norm(weight, 'fro'))
# print(weight_list)
    

[Parameter containing:
tensor([[ 0.0122, -0.0108,  0.0121,  ...,  0.0094,  0.0129,  0.0107],
        [ 0.0107, -0.0094,  0.0099,  ...,  0.0078,  0.0114,  0.0088],
        [ 0.0086, -0.0070,  0.0085,  ...,  0.0084,  0.0094,  0.0087],
        ...,
        [ 0.1051, -0.0907, -0.0344,  ..., -0.0585,  0.1269, -0.0502],
        [-0.1164,  0.1081,  0.0940,  ...,  0.1183, -0.1404,  0.1182],
        [ 0.1026, -0.0877, -0.0278,  ..., -0.0501,  0.1228, -0.0449]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([[-0.0031,  0.0031,  0.0004,  ..., -0.0007, -0.0027, -0.0004],
        [-0.0032,  0.0032,  0.0017,  ...,  0.0012, -0.0030,  0.0014],
        [-0.0008,  0.0013,  0.0046,  ...,  0.0049, -0.0003,  0.0053],
        ...,
        [-0.0225,  0.0221,  0.0174,  ...,  0.0209, -0.0248,  0.0214],
        [ 0.0166, -0.0113,  0.0299,  ...,  0.0284,  0.0224,  0.0302],
        [-0.0228,  0.0227,  0.0193,  ...,  0.0236, -0.0244,  0.0237]],
       device='cuda:0', requires_grad=True

In [278]:
# generate policy weights
a_mean_weights = pg_mu(act_dim, a_mu_feature)
a_logstd_weights = pg_logstd(act_dim, a_logstd_feature)
print(a_mean_weights.shape)
print(a_logstd_weights.shape)



torch.Size([64, 32, 6])
torch.Size([64, 32, 6])


In [279]:
a_mean_weights
for i in a_mean_weights:
    for j in i:
        print(j)


tensor([-0.1399, -0.1831, -0.1942, -0.1954, -0.1940, -0.1923], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.1559, -0.2181, -0.2421, -0.2508, -0.2537, -0.2544], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.2902, -0.3334, -0.3283, -0.3192, -0.3131, -0.3097], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.0017, -0.0039, -0.0053, -0.0062, -0.0067, -0.0069], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.2497, -0.3103, -0.3214, -0.3213, -0.3196, -0.3183], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.0009, 0.0033, 0.0051, 0.0061, 0.0067, 0.0070], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.2061, -0.2726, -0.2937, -0.3001, -0.3020, -0.3024], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.0003, 0.0028, 0.0048, 0.0059, 0.0066, 0.0070], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.0466, -0.1140, -0.1614, -0.1898, -0.2058, -0.2144], device='cuda:0',
       grad_fn=<Unbi

In [280]:
torch.norm(a_mean_weights, 'fro')

tensor(15.1747, device='cuda:0', grad_fn=<NormBackward1>)

In [281]:
a_logstd_weights
for i in a_logstd_weights:
    for j in i:
        print(j)


tensor([0.1541, 0.2174, 0.2373, 0.2396, 0.2363, 0.2322], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.1410, -0.2005, -0.2206, -0.2243, -0.2223, -0.2191], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.1110, -0.1879, -0.2384, -0.2701, -0.2894, -0.3009], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.1489, 0.2438, 0.3017, 0.3360, 0.3557, 0.3668], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.2799, 0.4410, 0.5222, 0.5604, 0.5777, 0.5855], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.1896, 0.3012, 0.3629, 0.3957, 0.4125, 0.4209], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.1537, -0.2160, -0.2350, -0.2366, -0.2328, -0.2283], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([0.1312, 0.2163, 0.2693, 0.3012, 0.3199, 0.3307], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tensor([-0.1317, -0.2119, -0.2592, -0.2862, -0.3014, -0.3097], device='cuda:0',
       grad_fn=<UnbindBackward0>)
tens

In [211]:
torch.norm(a_logstd_weights, 'fro')

tensor(16.4787, device='cuda:0', grad_fn=<NormBackward1>)

In [282]:
a_mu = bmm_output(a_mean_weights, a_mu_feature)
a_logstd = bmm_output(a_logstd_weights, a_logstd_feature)


In [283]:
print(a_mu)
print(a_logstd)


tensor([[-0.7333, -0.7753, -0.7045, -0.6430, -0.6041, -0.5818],
        [-0.6564, -0.7348, -0.7023, -0.6657, -0.6415, -0.6275],
        [-0.4083, -0.5002, -0.5096, -0.5030, -0.4963, -0.4920],
        [-0.4931, -0.5155, -0.4543, -0.4003, -0.3653, -0.3449],
        [-0.7116, -0.7266, -0.6354, -0.5607, -0.5140, -0.4872],
        [-0.3321, -0.3405, -0.2888, -0.2432, -0.2132, -0.1955],
        [-0.5474, -0.6013, -0.5603, -0.5189, -0.4915, -0.4756],
        [ 0.1022,  0.2289,  0.3267,  0.3911,  0.4303,  0.4532],
        [-0.4233, -0.4187, -0.3432, -0.2805, -0.2403, -0.2168],
        [-0.6553, -0.6619, -0.5690, -0.4934, -0.4458, -0.4184],
        [-0.4598, -0.5188, -0.4930, -0.4625, -0.4416, -0.4292],
        [-0.8273, -0.8160, -0.6926, -0.5971, -0.5383, -0.5048],
        [-0.4670, -0.5516, -0.5467, -0.5297, -0.5168, -0.5090],
        [-0.2641, -0.2521, -0.1929, -0.1438, -0.1118, -0.0929],
        [-0.7788, -0.8018, -0.7110, -0.6366, -0.5901, -0.5636],
        [ 0.1670,  0.2954,  0.3810,  0.4

In [284]:
# a_logstd = torch.clamp(a_logstd , min=-20, max=1)
actor_std = torch.exp(a_logstd)
print(actor_std)


tensor([[6.0422e-01, 4.7075e-01, 4.2167e-01, 4.0390e-01, 3.9827e-01, 3.9719e-01],
        [4.7698e-01, 3.2362e-01, 2.6902e-01, 2.4779e-01, 2.3940e-01, 2.3619e-01],
        [5.7221e-01, 4.2969e-01, 3.7610e-01, 3.5527e-01, 3.4746e-01, 3.4486e-01],
        [4.2648e-01, 2.6484e-01, 2.0692e-01, 1.8307e-01, 1.7251e-01, 1.6768e-01],
        [5.1321e-01, 3.5688e-01, 2.9717e-01, 2.7219e-01, 2.6133e-01, 2.5659e-01],
        [4.6083e-01, 3.0166e-01, 2.4357e-01, 2.1976e-01, 2.0947e-01, 2.0494e-01],
        [6.8365e-01, 5.6232e-01, 5.1280e-01, 4.9257e-01, 4.8472e-01, 4.8205e-01],
        [4.4691e-01, 2.9090e-01, 2.3601e-01, 2.1442e-01, 2.0555e-01, 2.0193e-01],
        [4.9695e-01, 3.4222e-01, 2.8512e-01, 2.6209e-01, 2.5253e-01, 2.4860e-01],
        [6.3260e-01, 5.0108e-01, 4.4992e-01, 4.2990e-01, 4.2255e-01, 4.2029e-01],
        [5.4119e-01, 3.9543e-01, 3.4216e-01, 3.2188e-01, 3.1442e-01, 3.1203e-01],
        [7.1558e-01, 5.8399e-01, 5.1906e-01, 4.8578e-01, 4.6839e-01, 4.5925e-01],
        [4.9334e

In [285]:
probs = Normal(a_mu, actor_std)
print(probs)
if action is None:
    action = probs.sample()
print(action)


Normal(loc: torch.Size([64, 6]), scale: torch.Size([64, 6]))
tensor([[-0.3394, -0.5920, -0.2063, -0.4934, -0.8681, -0.4183],
        [-0.5186, -0.2264, -0.5929,  0.1104, -0.2527,  0.1605],
        [-0.7078, -0.4836, -0.3255, -0.4236,  0.0553, -0.6947],
        [-1.4904, -0.6485, -0.3887, -0.6916, -0.0295, -0.4688],
        [-0.3576, -0.3256, -0.8207, -0.1758,  0.0187, -0.6320],
        [-0.4053, -0.9242, -0.1991, -0.3885, -0.4958, -0.4437],
        [-1.6029, -0.5038, -0.6825, -0.3779, -0.8993, -0.6861],
        [ 0.2067, -0.2916, -0.2376, -0.1605, -0.8595,  0.2472],
        [-0.8773, -1.2684, -0.5819, -1.1523, -0.5987, -0.6717],
        [-1.7136, -0.4277, -1.0313, -1.4661, -1.2854,  0.1506],
        [-0.0188, -0.2449, -0.2226, -0.6569, -0.2207, -0.6551],
        [-0.9993, -0.8457, -1.0763, -0.2536, -1.1182, -0.9338],
        [-1.5159,  0.3537, -0.2080, -0.3194, -0.2398, -0.8993],
        [-0.7828,  0.5601, -0.2129,  0.1556, -0.8783, -0.2803],
        [-0.7300, -1.2074, -0.7534, -0.8798

In [288]:
probs.log_prob(mb_actions)

tensor([[-1.6622e+00, -6.9687e-01, -1.6628e-01, -5.6633e-01, -6.3491e-03,
         -3.6964e-01],
        [-1.2503e+00, -7.7126e-01,  3.9163e-01,  3.5401e-01, -1.3197e+00,
          3.6463e-01],
        [-4.7693e-01, -9.4038e-01,  5.1428e-02, -1.8342e-01, -9.4018e-01,
          5.1669e-02],
        [-1.7066e+00, -9.1081e-01,  2.3264e-01,  7.5345e-01, -2.2909e+00,
          7.8571e-01],
        [-7.8186e-01, -5.1392e-01, -1.3948e+00,  3.5420e-01,  3.7666e-01,
          3.8416e-01],
        [-1.3971e+00, -1.1772e-01,  1.6078e-01, -5.8009e-01, -1.9443e+00,
          6.6509e-01],
        [-6.6996e-01, -3.4908e-01, -1.0389e+00, -2.8562e-01, -7.8988e-01,
         -1.9186e-01],
        [-2.9118e-01, -1.3679e+00,  4.1666e-01,  4.7546e-01, -5.1435e-01,
          6.0194e-01],
        [-4.4954e-01, -3.4726e-01, -4.2446e-01, -3.4064e-01, -8.5454e-02,
         -1.0870e-01],
        [-6.5528e-01, -1.1322e+00, -2.3987e-01, -1.5661e+00, -5.8388e-02,
         -1.5931e-01],
        [-3.2840e-01, -1.3434e

In [289]:
probs.entropy()

tensor([[ 9.1512e-01,  6.6551e-01,  5.5540e-01,  5.1234e-01,  4.9832e-01,
          4.9561e-01],
        [ 6.7866e-01,  2.9074e-01,  1.0597e-01,  2.3780e-02, -1.0697e-02,
         -2.4178e-02],
        [ 8.6069e-01,  5.7425e-01,  4.4104e-01,  3.8406e-01,  3.6182e-01,
          3.5434e-01],
        [ 5.6675e-01,  9.0320e-02, -1.5649e-01, -2.7897e-01, -3.3837e-01,
         -3.6676e-01],
        [ 7.5186e-01,  3.8859e-01,  2.0550e-01,  1.1769e-01,  7.6981e-02,
          5.8663e-02],
        [ 6.4422e-01,  2.2049e-01,  6.5876e-03, -9.6267e-02, -1.4424e-01,
         -1.6609e-01],
        [ 1.0386e+00,  8.4326e-01,  7.5108e-01,  7.1082e-01,  6.9475e-01,
          6.8922e-01],
        [ 6.1355e-01,  1.8416e-01, -2.4926e-02, -1.2090e-01, -1.6311e-01,
         -1.8092e-01],
        [ 7.1968e-01,  3.4663e-01,  1.6409e-01,  7.9863e-02,  4.2702e-02,
          2.7046e-02],
        [ 9.6102e-01,  7.2794e-01,  6.2026e-01,  5.7475e-01,  5.5750e-01,
          5.5213e-01],
        [ 8.0496e-01,  4.9115e

In [255]:
ctrl_cost = -torch.sum(action * action) * 0.1
ctrl_cost 

tensor(-4.5512, device='cuda:0')